# Clear Sky Examples

## Overview of Functionality

The captest clear sky functionality is based entirely on wrapping and integrating the clear sky modeling capabilities of the [pvlib-python package](https://pvlib-python.readthedocs.io/en/latest/index.html).  The primary intent of this functionality is to provide the option to easily calculate and plot modeled clear sky data as part of the workflow of loading, visualizing, and validating data from pyranometers.

When setting clear_sky to True when calling the CapData load_data method, the csky function will be called when loading data and the modeled clear sky POA and GHI data will be added to the dataframe.  The plot method will detect these columns and plot them as dashed lines with the measured irradiance data.

Below is a basic example of this functionality using data from the NREL Solar Radiation Research Library.

Andreas, A.; Stoffel, T.; (1981). NREL Solar Radiation Research Laboratory (SRRL): Baseline
Measurement System (BMS); Golden, Colorado (Data); NREL Report No. DA-5500-56488.
http://dx.doi.org/10.5439/1052221 

In [ ]:
import pandas as pd
import captest as pvc

from bokeh.io import output_notebook
output_notebook()

In [ ]:
meas_nrel = pvc.CapData()

In [ ]:
loc = {'latitude': 39.742, 'longitude': -105.18, 'altitude': 1828.8, 'tz': 'Etc/GMT+7'}
sys = {'surface_tilt': 40, 'surface_azimuth': 180, 'albedo': 0.2}

In [ ]:
meas_nrel.load_data(fname='nrel_data.csv', source='AlsoEnergy', clear_sky=True, loc=loc, sys=sys)

In [ ]:
meas_nrel.plot(ncols=1, width=800, merge_grps=['irr'], 
               subset=['irr_comb'])

Viewing a few rows of the dataframe within the CapData object shows that the modeled clear sky POA and GHI irradiance have been added as columns when loading the data.

In [ ]:
meas_nrel.df['3/10/2019 12:00':'3/10/2019 12:03']

## Explanation of csky Functions

The clear sky functionality is based around 4 top-level functions:
- pvlib_location
- pvlib_system
- get_tz_index
- csky

### Location and System wrappers: `pvlib_location` and `pvlib_system`

The first two, pvlib_location and pvlib_system, are simply wrappers that generate pvlib location and system objects from a dictionary.  The intent of providing these wrapper functions is to allow the captest user to be able to simply specify a dictionary for each without needing to import the entire pvlib package or know where the `Location` and system objects are within pvlib.  

The location dictionary defined in the example can be used with the `pvlib_location` function to create a pvlib location object.

In [ ]:
loc

In [ ]:
bms_location = pvc.pvlib_location(loc)

In [ ]:
type(bms_location)

In [ ]:
bms_location

Similarly, the `pvlib_system` function can be used to generate a pvlib system object.

Creating a pvlib ModelChain object requires providing a system object with module and inverter parameters defined.  The captest pvlib_system function provides arbitrary module and inverter parameters, so the captest user does not need to find and specify these.  The module and inverter parameters are not used when calculating the clear sky data.

The `pvlib_system` function also determines from the keywords of the passed dictionary if a `PVSystem` or `SingleAxisTracker` pvlib object should be created.  There is a tracking system example below.

In [ ]:
sys

In [ ]:
bms_system = pvc.pvlib_system(sys)

In [ ]:
type(bms_system)

In [ ]:
bms_system

In [ ]:
bms_system.module_parameters.head()

In [ ]:
bms_system.inverter_parameters.head()

### Get Timezone Index `get_tz_index`

The pvlib methods used to calculate clear sky irradiance require a time-zone aware datetime index as an argument.  The get_tz_index function returns a time-zone aware datetime index given a datetime index or a dataframe.  If the passed argument is already timezone aware, the index will be used as is; otherwise, the timezone will be set using the timezone in the location dictionary.

**Be sure the timezone matches the timezone of the measured data.**

For example, the time stamps of the example data from NREL are in MST and do not adjust for daylight savings, so specifying a timezone that does follow daylight savings, like 'America/Denver' will cause an error. Please refer to the pvlib documentaiton, which has a helpful [section on timezones](https://pvlib-python.readthedocs.io/en/latest/timetimezones.html), for more information.

The example usage of `get_tz_index` below matches the usage within the csky function in the example above. The function returns a time-zone aware datetime index.

In [ ]:
print(meas_nrel.df.index.tz)

In [ ]:
pvc.get_tz_index(meas_nrel.df, loc)

Passing the datetime index rather than the dataframe will return the same time-zone aware index.

In [ ]:
pvc.get_tz_index(meas_nrel.df.index, loc)

The below example shows the behavior if the time_source, dataframe or datetime index, already has a timezone when passed to `get_tz_index`. The timezone of the time_source is used and a warning is raised to alert the user that the timezone of the time_source and the passed location dictionary do not match.

In [ ]:
df = meas_nrel.df.copy()

In [ ]:
df.index = df.index.tz_localize('America/Caracas')

In [ ]:
loc

In [ ]:
pvc.get_tz_index(df, loc)

### Clear Sky Function `csky`

The clear sky function calls `pvlib_location`, `pvlib_system`, and `get_tz_index` and genarates pvlib objects to calculate modeled clear sky GHI and POA.  The essential functionality is shown in the example where the clear sky POA and GHI are added to the imported data.  `csky` can also return any componenet of the modeled csky data directly as shown below.

In [ ]:
poa_ghi = pvc.csky(meas_nrel.df, loc=loc, sys=sys, concat=False, output='both')
poa_ghi['3/10/2019 12:00':'3/10/2019 12:05']

In [ ]:
all_irrad_comp = pvc.csky(meas_nrel.df, loc=loc, sys=sys, concat=False, output='all')
all_irrad_comp['3/10/2019 12:00':'3/10/2019 12:05']

## Tracking System Example

It is possible to calculate the POA irradiance for a single-axis tracking system by defining location dictionary with tracker specific values.

In [ ]:
meas_nrel_trck = pvc.CapData()

In [ ]:
loc = {'latitude': 39.742, 'longitude': -105.18, 'altitude': 1828.8, 'tz': 'Etc/GMT+7'}
trck_sys = {'axis_tilt': 0, 'axis_azimuth': 0, 'max_angle': 52, 'backtrack': True, 'gcr': 0.2, 'albedo': 0.2}

In [ ]:
meas_nrel_trck.load_data(fname='nrel_data.csv', source='AlsoEnergy', clear_sky=True, loc=loc, sys=trck_sys)

In [ ]:
meas_nrel_trck.plot(ncols=1, width=800, merge_grps=['irr'], 
                    subset=['irr_comb'])